## Import packages

In [ ]:
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# set random seed
manual_seed = 999
random.seed(manual_seed)
torch.manual_seed(manual_seed)


## Setting some parameters

In [ ]:
batch_size = 128
image_size = 64
channels = 3
z_dim = 100
ngpu = 1

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

## The Data

In [ ]:
trans = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_set = dset.ImageFolder(root='../input/img_align_celeba/', transform=trans)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

print(len(train_set))


In [ ]:
batch = next(iter(train_loader))[0]

nrows=4
ncols=4
fig, axes = plt.subplots(nrows, ncols, sharex=True, sharey=True, squeeze=True)
for i in range(nrows):
    for j in range(ncols):
        img = batch[nrows * i + j].numpy()
        img = img.transpose((1, 2, 0))
        axes[i, j].imshow(img)

plt.show()

## Weight Initialization

In [ ]:
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


## The Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.network = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            # state size = 512*4*4
            
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # state size = 256*8*8
            
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            # state size = 128*16*16
            
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # state size = 64*32*32
            
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh(),
            # state size = 3*64*64
        )
    
    def forward(self, x):
        return self.network(x)


### Initializing weights + GPU setup

Now, we can instantiate the generator and apply the `weights_init` function. Check out the printed model to see how the generator object is structured.

In [ ]:
# create the generator
netG = Generator().to(device)

if device.type == 'cuda':
    netG.to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda' and ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# apply the weight_init function to randomly initialize all the weights
netG.apply(weight_init)

# print the model
print(netG)

## The Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size = 128*32*32
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size = 128*16*16
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size = 256*8*8
            
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size = 512*4*4
            
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            # state size 1*1*1
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)


### Initializing weights + GPU setup

In [ ]:
netD = Discriminator().to(device)

if device.type == 'cuda' and ngpu > 1:
    netD = nn.DataParallel(netD, list(range(ngpu)))

# initializing the weights
netD.apply(weight_init)

print(netD)

## Loss Functions and Optimizers
<br/><br/>
With D and G setup, we can specify how they learn through the loss functions and optimizers. We will use the Binary Cross Entropy loss ([BCELoss](https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss)) function which is defined in PyTorch as:

> ℓ(x,y) = L = $\{l_1,…,l_N\}^⊤$, $\ln$ = $−[y_n⋅\log(x_n)+(1−y_n)⋅log(1−x_n)]$

Notice how this function provides the calculation of both log components in the objective function (i.e. `log(D(x))` and `log(1−D(G(z))))`. We can specify what part of the BCE equation to use with the y input. This is accomplished in the training loop which is coming up soon, but it is important to understand how we can choose which component we wish to calculate just by changing y (i.e. GT labels).
<br/><br/>
Next, we define our real label as 1 and the fake label as 0. These labels will be used when calculating the losses of D and G, and this is also the convention used in the original GAN paper. Finally, we set up two separate optimizers, one for D and one for G. As specified in the DCGAN paper, both are Adam optimizers with `learning rate = 0.0002` and `Beta1 = 0.5`. For keeping track of the generator’s learning progression, we will generate a fixed batch of latent vectors that are drawn from a Gaussian distribution (i.e. fixed_noise) . In the training loop, we will periodically input this fixed_noise into G, and over the iterations we will see images form out of the noise.

In [ ]:
# inititalize the BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
# the progression of the generator
fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_labels = 1
fake_labels = 0

# Setup Adam optimizers for both G and D
lr = 0.0002

g_optim = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
d_optim = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))


## Training
<br/><br/>
Finally, now that we have all of the parts of the GAN framework defined, we can train it. Be mindful that training GANs is somewhat of an art form, as incorrect hyperparameter settings lead to mode collapse with little explanation of what went wrong. Here, we will closely follow Algorithm 1 from Goodfellow’s paper, while abiding by some of the best practices shown in [**ganhacks**](https://github.com/soumith/ganhacks). Namely, we will “construct different mini-batches for real and fake” images, and also adjust G’s objective function to maximize `logD(G(z))`. Training is split up into two main parts.
<br/><br/>
<p class='h4'>Part 1 - Train the Discriminator</p>
<br/>
Recall, the goal of training the discriminator is to maximize the probability of correctly classifying a given input as real or fake. In terms of Goodfellow, we wish to “update the discriminator by ascending its stochastic gradient”. Practically, we want to maximize `log(D(x))+log(1−D(G(z)))`. Due to the separate mini-batch suggestion from ganhacks, we will calculate this in two steps. First, we will construct a batch of real samples from the training set, forward pass through D, calculate the loss `log(D(x))`, then calculate the gradients in a backward pass. Secondly, we will construct a batch of fake samples with the current generator, forward pass this batch through D, calculate the loss `log(1−D(G(z)))`, and accumulate the gradients with a backward pass. Now, with the gradients accumulated from both the all-real and all-fake batches, we call a step of the Discriminator’s optimizer.
<br/><br/>

<p class='h4'>Part 2 - Train the Generator</p>
<br/>
As stated in the original paper, we want to train the Generator by minimizing `log(1−D(G(z)))` in an effort to generate better fakes. As mentioned, this was shown by Goodfellow to not provide sufficient gradients, especially early in the learning process. As a fix, we instead wish to maximize `log(D(G(z)))`. In the code we accomplish this by: classifying the Generator output from Part 1 with the Discriminator, computing G’s loss using real labels as GT, computing G’s gradients in a backward pass, and finally updating G’s parameters with an optimizer step. It may seem counter-intuitive to use the real labels as GT labels for the loss function, but this allows us to use the `log(x)` part of the BCELoss (rather than the `log(1−x)` part) which is exactly what we want.
<br/><br/>

<p class='h4'>Part 3 - Showing some statistics</p>
<br/>
Finally, we will do some statistic reporting and at the end of each epoch we will push our fixed_noise batch through the generator to visually track the progress of G’s training. The training statistics reported are:

- **Loss_D** - discriminator loss calculated as the sum of losses for the all real and all fake batches `log(D(x))+log(D(G(z)))`.
- **Loss_G** - generator loss calculated as `log(D(G(z)))`
- **D(x)** - the average output (across the batch) of the discriminator for the all real batch. This should start close to 1 then theoretically converge to 0.5 when G gets better. Think about why this is.
- **D(G(z))** - average discriminator outputs for the all fake batch. The first number is before D is updated and the second number is after D is updated. These numbers should start near 0 and converge to 0.5 as G gets better. Think about why this is.

In [ ]:
# lists to keep track of progress

img_list = []
g_losses = []
d_losses = []
iters = 0
epochs = 30
print_every = 50
save_img_every = 500


In [ ]:
# training loop
print('starting training...')

for epoch in range(epochs):
    for i, data in enumerate(train_loader):
        
        ################################################################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))  #
        ################################################################
        
        # train with all-real batch
        netD.zero_grad()
        # Format batch
        real = data[0].to(device)
        b_size = real.size(0)
        label = torch.full((b_size,), real_labels, device=device)
        
        # forward pass real batch through D
        output = netD(real).view(-1)
        
        # calculate loss on all-real batch
        d_loss_real = criterion(output, label)
        
        # calculate gradients for D in backward pass
        d_loss_real.backward()
        d_x = output.mean().item()
        
        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, z_dim, 1, 1, device=device)
        # generate fake images with G
        fake = netG(noise)
        label.fill_(fake_labels)
        # classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # calculate D's loss on the all-fake batch
        d_loss_fake = criterion(output, label)
        # Calculate the gradients for this batch
        d_loss_fake.backward()
        d_g_z1 = output.mean().item()
        # add the gradients from the all-real and all-fake batches
        d_loss = d_loss_fake + d_loss_real
        # update D
        d_optim.step()
        
        ################################################
        # (2) Update G network: maximize log(D(G(z)))  #
        ################################################
        
        netG.zero_grad()
        label.fill_(real_labels)  # fake labels are real for generator cost
        
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        
        # Calculate G's loss based on this output
        g_loss = criterion(output, label)
        # Calculate gradients for G
        g_loss.backward()
        d_g_z2 = g_loss.mean().item()
        # Update G
        g_optim.step()
        
        if i % print_every == 0:
            print('[{}/{}][{}/{}]\tLoss_D: {:.4f}\tLoss_G: {:.4f}\tD(x): {:.4f}\tD(G(z)): {:.4f}'.format(
                epoch, epochs, i, len(train_loader), d_loss.item(), g_loss.item(), d_x, d_g_z1, d_g_z2))
        
        # save losses for plotting
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())
        
        # Output training stats
        if i % save_img_every == 0:
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

print('end of training...')


## Results
### Loss versus training iteration

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses,label="G")
plt.plot(d_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Visualization of G’s progression

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

### Real Images vs. Fake Images

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(train_loader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()
